In [83]:
import pandas as pd
import os

In [84]:
kaggle_file = os.path.join('.','resources','cars_kaggle.csv')
vfacts_file = os.path.join('.','resources','2012_JAN_VFACTS_NSW.csv')

In [85]:
vdf = pd.read_csv(vfacts_file)
kdf = pd.read_csv(kaggle_file)

In [86]:
kdf

,Make,Model,Type,Origin,DriveTrain,MSRP,Invoice,EngineSize,Cylinders,Horsepower,MPG_City,MPG_Highway,Weight,Wheelbase,Length
0,Acura,MDX,SUV,Asia,All,"$36,945","$33,337",3.5,6.0,265,17,23,4451,106,189
1,Acura,RSX Type S 2dr,Sedan,Asia,Front,"$23,820","$21,761",2.0,4.0,200,24,31,2778,101,172
2,Acura,TSX 4dr,Sedan,Asia,Front,"$26,990","$24,647",2.4,4.0,200,22,29,3230,105,183
3,Acura,TL 4dr,Sedan,Asia,Front,"$33,195","$30,299",3.2,6.0,270,20,28,3575,108,186
4,Acura,3.5 RL 4dr,Sedan,Asia,Front,"$43,755","$39,014",3.5,6.0,225,18,24,3880,115,197
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
423,Volvo,C70 LPT convertible 2dr,Sedan,Europe,Front,"$40,565","$38,203",2.4,5.0,197,21,28,3450,105,186
424,Volvo,C70 HPT convertible 2dr,Sedan,Europe,Front,"$42,565","$40,083",2.3,5.0,242,20,26,3450,105,186
425,Volvo,S80 T6 4dr,Sedan,Europe,Front,"$45,210","$42,573",2.9,6.0,268,19,26,3653,110,190
426,Volvo,V40,Wagon,Europe,Front,"$26,135","$24,641",1.9,4.0,170,22,29,2822,101,180


In [87]:
def clean_currency(x):
    """ If the value is a string, then remove currency symbol and delimiters
    otherwise, the value is numeric and can be converted
    """
    if isinstance(x, str):
        return(x.replace('$', '').replace(',', ''))
    return(x)

In [88]:
kdf['MSRP'] = kdf['MSRP'].apply(clean_currency).astype('float')
kdf[['Make','Origin','MSRP']]

,Make,Origin,MSRP
0,Acura,Asia,36945.0
1,Acura,Asia,23820.0
2,Acura,Asia,26990.0
3,Acura,Asia,33195.0
4,Acura,Asia,43755.0
...,...,...,...
423,Volvo,Europe,40565.0
424,Volvo,Europe,42565.0
425,Volvo,Europe,45210.0
426,Volvo,Europe,26135.0


In [89]:
#reduce columns down to the ones we care about
cols = ['Make','Origin','MSRP','EngineSize','Cylinders','Horsepower','MPG_City','MPG_Highway','Weight','Wheelbase','Length']
kdf = kdf[cols]

In [90]:
#cols to aggregate
agg_cols = ['Origin','MSRP','EngineSize','Cylinders','Horsepower','MPG_City','MPG_Highway','Weight','Wheelbase','Length']
kdf_max = kdf.groupby(['Make']).max()[agg_cols].add_prefix("MAX_")
agg_cols.remove('Origin')
kdf_min = kdf.groupby(['Make']).min()[agg_cols].add_prefix("MIN_")
kdf_avg = kdf.groupby(['Make']).mean()[agg_cols].add_prefix("AVG_")
kdf_agg = kdf_max.join(kdf_min).join(kdf_avg).copy()
kdf_agg['car_count'] = kdf.groupby('Make').count()['Length']

In [91]:
#Merge the dataframes together and exclude anything that didnt have a Make in both tables so we only get fully populated data
final_df = pd.merge(kdf_agg,vdf,on="Make",how="inner")

In [92]:
final_df

,Make,MAX_Origin,MAX_MSRP,MAX_EngineSize,MAX_Cylinders,MAX_Horsepower,MAX_MPG_City,MAX_MPG_Highway,MAX_Weight,MAX_Wheelbase,...,AVG_Cylinders,AVG_Horsepower,AVG_MPG_City,AVG_MPG_Highway,AVG_Weight,AVG_Wheelbase,AVG_Length,car_count,Passenger_Volume,SUV_Volume
0,Audi,Europe,84600.0,4.2,8.0,450,23,31,4399,121,...,6.105263,250.789474,18.473684,25.789474,3700.631579,105.315789,181.421053,19,240,220
1,BMW,Europe,73195.0,4.4,8.0,333,21,30,4824,123,...,6.400000,241.450000,18.700000,27.000000,3611.350000,108.950000,180.100000,20,239,158
2,Chrysler,USA,38380.0,3.8,6.0,255,22,30,4675,119,...,5.333333,201.133333,19.866667,27.333333,3534.333333,109.200000,190.000000,15,2,0
3,Dodge,USA,81795.0,8.3,10.0,500,29,36,4987,131,...,5.692308,209.692308,19.384615,26.230769,3638.769231,112.769231,194.230769,13,32,3
4,Ford,USA,41475.0,6.8,10.0,310,27,36,7190,137,...,6.260870,197.869565,19.260870,25.739130,3748.913043,111.434783,191.913043,23,1054,268
5,Honda,Asia,33260.0,3.5,6.0,240,60,66,4387,118,...,4.529412,169.411765,27.823529,34.000000,3101.000000,104.411765,179.294118,17,486,64
6,Hyundai,Asia,26189.0,3.5,6.0,194,29,34,3651,108,...,5.000000,149.666667,23.000000,29.916667,2930.000000,102.333333,178.583333,12,1441,276
7,Jaguar,Europe,86995.0,4.2,8.0,390,18,28,4046,119,...,7.500000,307.000000,17.500000,25.583333,3821.750000,110.333333,191.000000,12,28,0
8,Jeep,USA,27905.0,4.0,6.0,195,20,24,3826,106,...,5.333333,178.333333,17.333333,21.333333,3730.333333,101.000000,168.333333,3,0,302
9,Kia,Asia,26000.0,3.5,6.0,195,26,33,4802,115,...,4.727273,143.090909,21.909091,28.818182,3167.909091,102.909091,179.727273,11,617,152


In [93]:
final_df.insert(0,'id',range(1,1 + len(final_df)))

In [94]:
#Make the tables to export into SQL
final_df['total_vol'] = final_df[['Passenger_Volume','SUV_Volume']].sum(axis=1)
rename_dict = {
    'Make':'make',
    'MAX_Origin':'origin',
    'MAX_MSRP':'max_msrp',
    'MAX_EngineSize':'max_engine_size',
    'MAX_Cylinders':'max_cylinders',
    'MAX_Horsepower':'max_horsepower',
    'MAX_MPG_City':'max_mpg_city',
    'MAX_MPG_Highway':'max_mpg_highway',
    'MAX_Weight':'max_weight',
    'MAX_Wheelbase':'max_wheelbase',
    'MAX_Length':'max_length',
    'MIN_MSRP':'min_msrp',
    'MIN_EngineSize':'min_engine_size',
    'MIN_Cylinders':'min_cylinders',
    'MIN_Horsepower':'min_horsepower',
    'MIN_MPG_City':'min_mpg_city',
    'MIN_MPG_Highway':'min_mpg_highway',
    'MIN_Weight':'min_weight',
    'MIN_Wheelbase':'min_wheelbase',
    'MIN_Length':'min_length',
    'AVG_MSRP':'avg_msrp',
    'AVG_EngineSize':'avg_engine_size',
    'AVG_Cylinders':'avg_cylinders',
    'AVG_Horsepower':'avg_horsepower',
    'AVG_MPG_City':'avg_mpg_city',
    'AVG_MPG_Highway':'avg_mpg_highway',
    'AVG_Weight':'avg_weight',
    'AVG_Wheelbase':'avg_wheelbase',
    'AVG_Length':'avg_length',
    'Passenger_Volume':'passenger_vol',
    'SUV_Volume':'suv_vol'}
final_df.rename(columns=rename_dict,inplace=True)


In [95]:
#List of all the tables we want to create
make_sql_list = ['id','make','origin','car_count']
pricing_spec_list = ['id','min_msrp','avg_msrp','max_msrp']
engine_spec_list = ['id','min_engine_size','avg_engine_size','max_engine_size','min_cylinders','avg_cylinders','max_cylinders','min_horsepower','avg_horsepower','max_horsepower']
fuel_spec_list = ['id','min_mpg_city','avg_mpg_city','max_mpg_city','min_mpg_highway','avg_mpg_highway','max_mpg_highway']
chassis_spec_list = ['id','min_weight','avg_weight','max_weight','min_wheelbase','avg_wheelbase','max_wheelbase','min_length','avg_length','max_length']
sales_data_list = ['id','passenger_vol','suv_vol','total_vol']

In [96]:
from sqlalchemy import create_engine 

In [97]:
conn = "postgres:testing@localhost:5432/cars_db"
engine = create_engine(f'postgresql://{conn}')

In [98]:
engine.table_names()

['fuel_spec',
 'chassis_spec',
 'sales_data',
 'makes',
 'pricing_spec',
 'engine_spec']

In [99]:
final_df[make_sql_list].to_sql(name='makes',con=engine,if_exists='append',index=False)

In [100]:
pd.read_sql_query('select * from makes limit 5', con=engine)

,id,make,origin,car_count
0,1,Audi,Europe,19
1,2,BMW,Europe,20
2,3,Chrysler,USA,15
3,4,Dodge,USA,13
4,5,Ford,USA,23


In [101]:
final_df[pricing_spec_list].to_sql(name='pricing_spec',con=engine,if_exists='append',index=False)
final_df[engine_spec_list].to_sql(name='engine_spec',con=engine,if_exists='append',index=False)
final_df[fuel_spec_list].to_sql(name='fuel_spec',con=engine,if_exists='append',index=False)
final_df[chassis_spec_list].to_sql(name='chassis_spec',con=engine,if_exists='append',index=False)
final_df[sales_data_list].to_sql(name='sales_data',con=engine,if_exists='append',index=False)